In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
import timm
from timm.loss import LabelSmoothingCrossEntropy 
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from tqdm import tqdm
import time
import copy

In [2]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return val_loader, test_loader, len(val_data), len(test_data)

In [4]:
#dataset_path = 'C:/Users/CON3ADOLEA/Documents/deit-main/data'
dataset_path = 'M:/VieApp/biophilic_training_dataset'

In [5]:
(train_loader, train_data_len) = get_data_loaders(dataset_path, 128, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 32, train=False)

In [6]:
classes = train_loader.dataset.classes
print(classes, len(classes))

['awe', 'biomorphic_shapes', 'complexity_in_order', 'connection_with_nature', 'mystery', 'natural_materials', 'natural_organisation', 'presence_of_animals', 'presence_of_plants_or_fungi', 'presence_of_water', 'refuge', 'risk', 'unimpeded_views', 'varying_light'] 14


In [7]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [8]:
print(len(train_loader), len(val_loader), len(test_loader))

82 132 132


In [9]:
print(train_data_len, valid_data_len, test_data_len)

10373 4200 4200


In [10]:
#code for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
model = torch.hub.load('facebookresearch/deit:main', 'deit_tiny_patch16_224', pretrained =True)

Using cache found in C:\Users\CON3ADOLEA/.cache\torch\hub\facebookresearch_deit_main


In [12]:
for param in model.parameters(): #freeze model
    param.requires_grad = False

n_inputs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(n_inputs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(classes))
)
model = model.to(device)
#print(model.head)

In [13]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer = optim.Adam(model.head.parameters(), lr=0.001)

In [14]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=1000): #epoch 300 = 33.65 train and val=17.62%
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [54]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler) # now it is a lot faster

Epoch 0/299
----------


100%|██████████| 82/82 [02:44<00:00,  2.00s/it]


train Loss: 2.3116 Acc: 0.2176


100%|██████████| 132/132 [06:11<00:00,  2.81s/it]


val Loss: 2.4992 Acc: 0.1750

Epoch 1/299
----------


...

val Loss: 2.6712 Acc: 0.1610

Epoch 299/299
----------


100%|██████████| 82/82 [02:38<00:00,  1.93s/it]


train Loss: 1.9172 Acc: 0.3365


100%|██████████| 132/132 [05:36<00:00,  2.55s/it]

val Loss: 2.6714 Acc: 0.1614

Training complete in 2443m 6s
Best Val Acc: 0.1762


In [17]:
#second test 
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler) # now it is a lot faster

Epoch 0/999
----------


100%|██████████| 82/82 [02:52<00:00,  2.10s/it]


train Loss: 2.5278 Acc: 0.1547


100%|██████████| 132/132 [06:16<00:00,  2.86s/it]


val Loss: 2.5130 Acc: 0.1536

Epoch 1/999
----------


...

val Loss: 2.8086 Acc: 0.1560

Epoch 999/999
----------


100%|██████████| 82/82 [02:47<00:00,  2.04s/it]


train Loss: 1.8284 Acc: 0.3522


100%|██████████| 132/132 [06:40<00:00,  3.04s/it]

val Loss: 2.8086 Acc: 0.1560

Training complete in 9606m 7s
Best Val Acc: 0.1724


In [17]:
#old training code #model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler) # now it is a lot faster

Epoch 0/99
----------


100%|██████████| 27/27 [01:37<00:00,  3.62s/it]


train Loss: 1.4985 Acc: 0.5711


100%|██████████| 106/106 [01:43<00:00,  1.02it/s]


val Loss: 1.3951 Acc: 0.6085

Epoch 1/99
----------


...

val Loss: 1.1066 Acc: 0.6830

Epoch 99/99
----------


100%|██████████| 27/27 [01:39<00:00,  3.70s/it]


train Loss: 1.1911 Acc: 0.6664


100%|██████████| 106/106 [01:33<00:00,  1.14it/s]

val Loss: 1.1057 Acc: 0.6836

Training complete in 309m 21s
Best Val Acc: 0.6842


In [ ]:
#testing model accuracy

In [18]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model.eval()

for data, target in tqdm(test_loader):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == 32:
        for i in range(32):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

test_loss = test_loss / test_data_len
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print("Test Accuracy of %5s: %2d%% (%2d/%2d)" % (
            classes[i], 100*class_correct[i]/class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))
print("Test Accuracy of %2d%% (%2d/%2d)" % (
            100*np.sum(class_correct)/np.sum(class_total), np.sum(class_correct), np.sum(class_total)
        ))


100%|██████████| 106/106 [01:34<00:00,  1.12it/s]

Test Loss: 0.0033
Test Accuracy of Varying_light: 69% (108/156)
Test Accuracy of   awe: 73% (259/353)
Test Accuracy of biomorphic shapes: 58% (155/263)
Test Accuracy of complexity_in_order: 93% (186/199)
Test Accuracy of connection_with_nature: 78% (157/200)
Test Accuracy of mystery: 75% (151/200)
Test Accuracy of natural materials: 51% (171/333)
Test Accuracy of natural organisation: 61% (238/385)
Test Accuracy of presence of animals: 79% (166/208)
Test Accuracy of presence_of_plants_or_fungi: 52% (80/151)
Test Accuracy of presence_of_water: 70% (133/190)
Test Accuracy of refuge: 52% (96/183)
Test Accuracy of  risk: 74% (159/214)
Test Accuracy of unimpeded views: 73% (240/325)
Test Accuracy of 68% (2299/3360)


In [ ]:
#save model

In [55]:
# our model earns 93% test accuracy, which is very high. lets save it
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model.cpu(), example)
traced_script_module.save("biophilic_deit_2_33.65.pt")

In [20]:
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
])

In [32]:
from PIL import Image
#img ='C:/Users/CON3ADOLEA/Documents/deit-main/1921.90 - Portrait of a Young Woman.jpg'
img ='C:/Users/CON3ADOLEA/Documents/deit-main/2.jpg'

im = Image.open(img)

In [33]:
# transform the original image and add a batch dimension
imag = transform(im).unsqueeze(0)

# compute the predictions
out = traced_script_module(imag)

# and convert them into probabilities
scores = torch.nn.functional.softmax(out, dim=-1)[0]

# finally get the index of the prediction with highest score
topk_scores, topk_label = torch.topk(scores, k=5, dim=-1)

for i in range(5):
    pred_name =classes[topk_label[i]]
    print(f"Prediction index {i}: {pred_name:<25}, score: {topk_scores[i].item():.3f}")

Prediction index 0: awe                      , score: 0.932
Prediction index 1: Varying_light            , score: 0.027
Prediction index 2: risk                     , score: 0.017
Prediction index 3: natural materials        , score: 0.007
Prediction index 4: natural organisation     , score: 0.004
